In [1]:
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cmocean
import spatialpandas.dask 
import dask as da
import dask.distributed as dd
import dask.dataframe as dd
import timeit

from matplotlib.tri import Triangulation as mpl_delaunay  

In [2]:
%load_ext autoreload
%autoreload 2

from polyplot import poly_plot as pp

# Data (Small)

In [3]:
# GeoFlow Small (6,000 nodes)
ds_base_path = "/glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/small/"
ds_grid = xr.open_dataset(ds_base_path + "grid.nc")
ds_v1 = xr.open_dataset(ds_base_path + "v1.000000.nc")
ds_v2 = xr.open_dataset(ds_base_path + "v2.000000.nc")
ds_v3 = xr.open_dataset(ds_base_path + "v3.000000.nc")

In [4]:
# Specify Time and Elevation Slice
t, level = 0, 0

# Coordinates & Face Data
lon_cell = ds_grid['mesh_node_x'].values
lat_cell = ds_grid['mesh_node_y'].values
face_nodes = ds_grid['mesh_face_nodes'].values

# Set up Projection (experimental)
projection = ccrs.Robinson(central_longitude=0)
x_pcs, y_pcs, _ = projection.transform_points(ccrs.PlateCarree(), lon_cell, lat_cell).T


# Data Variable to Plot
v1 = ds_v1['v1'][t][level].values
v2 = ds_v2['v2'][t][level].values
v3 = ds_v3['v3'][t][level].values

## Test with different mem.

# Testing (Small)

In [8]:
def poly_plot_workflow():
    geoflow = pp(lon_cell, lat_cell, face_nodes)
    geoflow.set_data(v1)
    geoflow.to_poly_mesh()
    df = geoflow.fix_cells()

In [10]:
%timeit -r 10 -n 10 mpl_delaunay(lon_cell, lat_cell)

18.5 ms ± 407 µs per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
%timeit -r 10 -n 100 poly_plot_workflow()

11.4 ms ± 163 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


# Data (Large)

In [13]:
# GeoFlow Small (6,000 nodes)
ds_base_path = "/glade/p/cisl/vast/vapor/data/Source/UGRID/NOAA-geoflow/large/"
ds_grid = xr.open_dataset(ds_base_path + "grid.nc")
ds_v1 = xr.open_dataset(ds_base_path + "v1.000000.nc")
ds_v2 = xr.open_dataset(ds_base_path + "v2.000000.nc")
ds_v3 = xr.open_dataset(ds_base_path + "v3.000000.nc")

In [14]:
# Specify Time and Elevation Slice
t, level = 0, 0

# Coordinates & Face Data
lon_cell = ds_grid['mesh_node_x'].values
lat_cell = ds_grid['mesh_node_y'].values
face_nodes = ds_grid['mesh_face_nodes'].values

# Set up Projection (experimental)
projection = ccrs.Robinson(central_longitude=0)
x_pcs, y_pcs, _ = projection.transform_points(ccrs.PlateCarree(), lon_cell, lat_cell).T


# Data Variable to Plot
v1 = ds_v1['v1'][t][level].values
v2 = ds_v2['v2'][t][level].values
v3 = ds_v3['v3'][t][level].values

# Testing (Large)

In [17]:
def poly_plot_workflow():
    geoflow = pp(lon_cell, lat_cell, face_nodes)
    geoflow.set_data(v1)
    geoflow.to_poly_mesh()
    df = geoflow.fix_cells()

In [19]:
%timeit -r 2 -n 5 mpl_delaunay(lon_cell, lat_cell)

40.5 s ± 130 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [15]:
%%time
poly_plot_workflow()

KeyboardInterrupt: 